In [ ]:
# %pip install fast-langdetect wordsegment fastapi gpt4all
# Entorno tt_env

In [ ]:
# INSTALACIONES PARA NUEVO AMBIENTE
# %pip install wordsegment
# %pip install transformers datasets torch pandas scikit-learn
# %pip install sentencepiece
# %pip install protobuf
# %pip install ipywidgets
# %pip install 'accelerate>=0.26.0'

# RED NEURONAL

En esta sección debido al desorden se empieza a trabajar en el modelo para sugerir el nombre de código.

In [ ]:
# import pandas as pd

# file_path = "./dataset/java_dataset.csv"
# df = pd.read_csv(file_path, encoding="latin-1")


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    RobertaTokenizer,
    T5ForConditionalGeneration,
    # T5Tokenizer,
    Trainer,
    TrainingArguments,
)


In [3]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    RobertaTokenizer,
    T5ForConditionalGeneration,
    # T5Tokenizer,
    Trainer,
    TrainingArguments,
)

# 1. Cargar el dataset desde un archivo CSV
file_path = "./dataset/dataset_final.csv"
df = pd.read_csv(file_path, encoding="utf-8")

# Asegurar que las columnas clave existen
assert "code" in df.columns and "suggested_name" in df.columns, (
    "El CSV debe contener 'code' y 'suggested_name'"
)

# 2. Tokenización con CodeT5
# tokenizer = T5Tokenizer.from_pretrained("Salesforce/codet5-small")

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")


def preprocess_data(examples):
    """Función para tokenizar código y nombres sugeridos"""
    inputs = [f"Generate name: {code}" for code in examples["code"]]
    targets = examples["suggested_name"]

    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )
    labels = tokenizer(targets, max_length=20, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Convertir DataFrame a Dataset de Hugging Face
dataset = Dataset.from_pandas(df)

# Aplicar tokenización al dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# 3. Dividir en train/test
split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset, test_dataset = split["train"], split["test"]

# 4. Cargar el modelo CodeT5
# model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")
# model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")


# # 5. Configurar entrenamiento
# training_args = TrainingArguments(
#     output_dir="./codet5-fine-tuned",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     learning_rate=5e-5,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_total_limit=2,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=tokenizer,
# )

# # 6. Entrenar el modelo
# trainer.train()

model = T5ForConditionalGeneration.from_pretrained(
    "./codet5-fine-tuned/checkpoint-28122"
)

training_args = TrainingArguments(
    output_dir="./codet5-fine-tuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    resume_from_checkpoint=True,  # Habilitar reanudación
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

trainer.train(resume_from_checkpoint=True)  # Reanuda desde el último checkpoint

# 7. Guardar el modelo
model.save_pretrained("./codet5-fine-tuned")
tokenizer.save_pretrained("./codet5-fine-tuned")

# print("¡Entrenamiento completado y modelo guardado!")

AssertionError: El CSV debe contener 'code' y 'suggested_name'

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")

text = "def greet(user): print(f'hello <extra_id_0>!')"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate a single sequence
generated_ids = model.generate(input_ids, max_length=10)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
# this prints "user: {user.name}"


In [ ]:
import re

from wordsegment import load, segment

load()

In [ ]:
tipo_datos = [
    "boolean",
    "char",
    "byte",
    "short",
    "int",
    "long",
    "float",
    "double",
    "String",
    "Array",
]

In [ ]:
text = """
    package com.hmkcode;

import java.io.BufferedReader;
import java.io.FileNotFoundException;
import java.io.FileReader;
import java.io.IOException;

public class textfilereader {

	private BufferedReader buffer;
	private String currentLine = "";
	public textfilereader(){ 
		
	}
	
	public void open(String file){
		
		try {
			close();
			
			buffer = new BufferedReader(new FileReader(file));
			
		} catch (FileNotFoundException e1) {
			e1.printStackTrace();
	
		}
		
	}
	
	public void close(){
		
		try {
			if(buffer != null){
				buffer.close();
				buffer = null;
			}
		} catch (IOException e) {
			e.printStackTrace();
		}
		
	}
	
	public String readLine() throws Exception{
		if(buffer != null){
			currentLine = buffer.readLine();
			
			if(currentLine == null)
				close();
			
			return currentLine;
		}
		else
			throw new Exception("No file to read...");
	}
	
	public String getCurrent(){
		return this.currentLine;
	}
	
	public boolean isReadable(){
		return (buffer != null && this.currentLine != null);
	}
}
"""

In [ ]:
# Declaración de regex para uso general
class_starter = r"(class)\s([^\{]+)\{"
funciones_static = r"(static\s){1,1}.{0,}(){1,1}"
funciones_private = r"(private\s){1,1}.{0,}(){1,1}"

In [ ]:
class_coin = re.findall(pattern=class_starter, string=text)
class_coin

In [ ]:
# Comprobamos el nombramiento de la clase
if bool(class_coin):
    # Con una expresión regular verificamos el nombramiento de la clase y su respectivo nombre
    identificador = class_coin[0][0].strip()

    t_t = text.split("\n")
    n = len(t_t)

    # Iteramos por cada linea de código hasta que terminamos con el proceso de cambiar el nombre de la clase por la forma de cammel case
    for i in range(n):
        if identificador in t_t[i]:
            sep_l = t_t[i].split()
            n_2 = len(sep_l)
            for j in range(n_2):
                if sep_l[j] != identificador and sep_l[j] != "public":
                    sep_l[j] = separate_words(sep_l[j])
                    break
            sep_l = " ".join(sep_l)
            t_t[i] = sep_l
text = "\n".join(t_t)

# De momento solo se revisa si hay nombramiento de metodos privados
if "private" in text:
    coincidencias = re.findall(pattern=funciones_private, string=text)

    if bool(coincidencias):
        # print("segundo if")
        c = coincidencias[0][0].strip()
        t_l = text.split("\n")
        for i in range(len(t_l)):
            # print(c,line)
            if c in t_l[i]:
                # print("antes del title", t_l[i].strip())
                sep = t_l[i].split()
                # print(sep.index(c))
                for j in range(len(sep)):
                    if sep[j] != c and sep[j] not in tipo_datos:
                        sep[j] = separate_words(sep[j])
                        break
                sep = " ".join(sep)
                t_l[i] = sep
text = "\n".join(t_l)

In [ ]:
dtb_code = """
import java.sql.Connection;
import java.sql.DriverManager;
import java.sql.ResultSet;
import java.sql.SQLException;
import java.sql.Statement;

public class DatabaseConnection {
    private static final String URL = "jdbc:mysql://localhost:3306/tu_base_de_datos";
    private static final String USER = "tu_usuario";
    private static final String PASSWORD = "tu_contraseña";

    private Connection connection;

    public DatabaseConnection() {
        try {
            // Cargar el driver de MySQL
            Class.forName("com.mysql.cj.jdbc.Driver");
            // Establecer la conexión
            connection = DriverManager.getConnection(URL, USER, PASSWORD);
            System.out.println("Conexión exitosa a la base de datos.");
        } catch (ClassNotFoundException | SQLException e) {
            e.printStackTrace();
        }
    }

    public void ejecutarConsulta(String consulta) {
        try (Statement statement = connection.createStatement();
             ResultSet resultSet = statement.executeQuery(consulta)) {
            while (resultSet.next()) {
                System.out.println("Resultado: " + resultSet.getString(1));
            }
        } catch (SQLException e) {
            e.printStackTrace();
        }
    }

    public void cerrarConexion() {
        try {
            if (connection != null && !connection.isClosed()) {
                connection.close();
                System.out.println("Conexión cerrada.");
            }
        } catch (SQLException e) {
            e.printStackTrace();
        }
    }

    public static void main(String[] args) {
        DatabaseConnection db = new DatabaseConnection();
        db.ejecutarConsulta("SELECT * FROM tu_tabla");
        db.cerrarConexion();
    }
}

"""

In [ ]:
class_coin = re.findall(pattern=class_starter, string=dtb_code)
class_coin


In [ ]:
def is_camell_case(word: str) -> bool:
    """Función para determinar si la case ya esta en formato camell case, de ser así retorna un True

    Args:
        word (str): Nombre de la clase o el método

    Returns:
        bool: Valor booleano acorde a si es cammel case o no
    """
    word = word.strip()

    sep = segment(word)

    final = "".join([item.capitalize() for item in sep])

    if final.strip() == word:
        return True
    elif final.strip() != word:
        return False

In [ ]:
is_camell_case(class_coin[0][1].strip())

In [ ]:
from gpt4all import GPT4All

llm = GPT4All(
    model_name="Meta-Llama-3-8B-Instruct.Q4_0.gguf",
    model_path="./models",
    verbose=True,
    allow_download=False,
)

with llm.chat_session():
    print(
        llm.generate(
            f"Dime si este código de java cumple con las buenas prácticas de nombramiento del libro de clean code de Robert C. Martin: {dtb_code}, si las variables no cumplen con ser lo suficientemente descriptivas, retorna un diccionario de python donde las llaves sean los nombres de las variables actuales y el valor sea el nuevo nombre. No me expliques nada, devuelve unicamente el diccionario sin dar explicaciones ni agregar ni un comentario"
        )
    )

In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    # T5Tokenizer,
    RobertaTokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
)

# 1. Cargar el dataset desde el archivo CSV
df = pd.read_csv("./dataset/dataset_final.csv", encoding="utf-8")

# Se cambia el valor para trabajar con el
df["is_correct"] = df["is_correct"].astype(str)

df = df.dropna()  # Se eliminan 4 nulos, que son muy pocos comparados con los demás

# Asegurar que las columnas clave existen
assert (
    "code" in df.columns
    and "suggested_name" in df.columns
    and "type" in df.columns
    and "is_correct" in df.columns
), "El CSV debe contener 'code', 'suggested_name', 'type' e 'is_correct'"

# 2. Tokenización con CodeT5
# tokenizer = T5Tokenizer.from_pretrained("Salesforce/codet5-small")
# tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")


# def preprocess_data(examples):
#     """Función para tokenizar código y nombres sugeridos con tipo y corrección"""
#     inputs = [
#         f"Generate name [type: {typ}, correct: {corr}]: {code}"
#         for typ, corr, code in zip(
#             examples["type"], examples["is_correct"], examples["code"]
#         )
#     ]
#     targets = examples["suggested_name"]

#     model_inputs = tokenizer(
#         inputs, max_length=512, truncation=True, padding="max_length"
#     )
#     labels = tokenizer(targets, max_length=20, truncation=True, padding="max_length")

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs


# # Convertir DataFrame a Dataset de Hugging Face
# dataset = Dataset.from_pandas(df)

# # Aplicar tokenización al dataset
# tokenized_dataset = dataset.map(preprocess_data, batched=True)

# # 3. Dividir en train/test
# split = tokenized_dataset.train_test_split(test_size=0.1)
# train_dataset, test_dataset = split["train"], split["test"]

# # 4. Cargar el modelo CodeT5
# model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")
# model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")

# # 5. Configurar entrenamiento
# training_args = TrainingArguments(
#     output_dir="./codet5-fine-tuned",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     learning_rate=5e-5,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_total_limit=2,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=tokenizer,
# )

# # 6. Entrenar el modelo
# trainer.train()

# # 7. Guardar el modelo
# model.save_pretrained("./codet5-fine-tuned")
# tokenizer.save_pretrained("./codet5-fine-tuned")

# print("¡Entrenamiento completado y modelo guardado!")


In [ ]:
df["is_correct"] = df["is_correct"].astype(str)

In [ ]:
df.info()

In [ ]:
print(df.isnull().sum())  # Verifica cuántos valores nulos hay por columna


In [ ]:
df = df.dropna()  # Elimina filas con valores nulos
# O
# df = df.fillna("")  # Reemplaza nulos con una cadena vacía

In [ ]:
print(df.isnull().sum())


In [ ]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    RobertaTokenizer,  # T5Tokenizer,
    T5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
)

# 1. Cargar el dataset desde el archivo CSV
df = pd.read_csv("./dataset/dataset_final.csv", encoding="utf-8")

# Se cambia el valor para trabajar con el
df["is_correct"] = df["is_correct"].astype(str)
df = df.dropna()  # Se eliminan 4 nulos, que son muy pocos comparados con los demás

# Asegurar que las columnas clave existen
assert (
    "code" in df.columns
    and "suggested_name" in df.columns
    and "type" in df.columns
    and "is_correct" in df.columns
), "El CSV debe contener 'code', 'suggested_name', 'type' e 'is_correct'"

# 2. Tokenización con CodeT5
# tokenizer = T5Tokenizer.from_pretrained("Salesforce/codet5-small")
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")


def preprocess_data(examples):
    """Función para tokenizar código y nombres sugeridos con tipo y corrección"""
    inputs = [
        f"Generate name [type: {typ}, correct: {corr}]: {code}"
        for typ, corr, code in zip(
            examples["type"], examples["is_correct"], examples["code"]
        )
    ]
    targets = examples["suggested_name"]

    model_inputs = tokenizer(
        inputs, max_length=512, truncation=True, padding="max_length"
    )
    labels = tokenizer(targets, max_length=20, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Convertir DataFrame a Dataset de Hugging Face
dataset = Dataset.from_pandas(df)

# Aplicar tokenización al dataset
tokenized_dataset = dataset.map(preprocess_data, batched=True)

# 3. Dividir en train/test
split = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset, test_dataset = split["train"], split["test"]

# # 4. Cargar el modelo CodeT5
# model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-small")

# # 5. Configurar entrenamiento
# training_args = TrainingArguments(
#     output_dir="./codet5-fine-tuned",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     learning_rate=5e-5,
#     num_train_epochs=5,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=10,
#     save_total_limit=2,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     tokenizer=tokenizer,
# )

# # 6. Entrenar el modelo
# trainer.train()

model = T5ForConditionalGeneration.from_pretrained(
    "./codet5-fine-tuned/checkpoint-28122"
)

training_args = TrainingArguments(
    output_dir="./codet5-fine-tuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    resume_from_checkpoint=True,  # Habilitar reanudación
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

trainer.train(resume_from_checkpoint=True)  # Reanuda desde el último checkpoint

# 7. Guardar el modelo
model.save_pretrained("./codet5-fine-tuned")
tokenizer.save_pretrained("./codet5-fine-tuned")

print("¡Entrenamiento completado y modelo guardado!")


Map: 100%|██████████| 83322/83322 [16:12<00:00, 85.65 examples/s] 
d:\Users\rod_e\miniconda3\envs\tt_env\lib\site-packages\transformers\training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\rod_e\AppData\Local\Temp\ipykernel_17244\1112393671.py:108: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
4,0.132100,0.107727
5,0.079900,0.107400


¡Entrenamiento completado y modelo guardado!


## Predicciones

Parte del código para crear las predicciones en base a inputs del código Java escrito por el usuario:

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration, T5Tokenizer

# Cargar el modelo y el tokenizador desde la carpeta donde lo guardaste
model_path = "./codet5-fine-tuned"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = RobertaTokenizer.from_pretrained(model_path)


In [3]:
def generate_suggested_name(code, type_label="class", is_correct="true"):
    """Genera un nombre sugerido para el código dado."""
    input_text = f"Generate name [type: {type_label}, correct: {is_correct}]: {code}"
    input_ids = tokenizer(
        input_text, return_tensors="pt", max_length=512, truncation=True
    ).input_ids

    # Generar la predicción
    output_ids = model.generate(input_ids, max_length=20, num_return_sequences=1)
    suggested_name = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return suggested_name


In [4]:
code = """ 
public class Jadx {
	private static final Logger LOG = LoggerFactory.getLogger(Jadx.class);

	private Jadx() {
	}

	public static List<IDexTreeVisitor> getPassesList(JadxArgs args) {
		switch (args.getDecompilationMode()) {
			case AUTO:
			case RESTRUCTURE:
				return getRegionsModePasses(args);
			case SIMPLE:
				return getSimpleModePasses(args);
			case FALLBACK:
				return getFallbackPassesList();
			default:
				throw new JadxRuntimeException("Unknown decompilation mode: " + args.getDecompilationMode());
		}
	}

	public static List<IDexTreeVisitor> getPreDecompilePassesList() {
		List<IDexTreeVisitor> passes = new ArrayList<>();
		passes.add(new SignatureProcessor());
		passes.add(new OverrideMethodVisitor());
		passes.add(new AddAndroidConstants());

		// rename and deobfuscation
		passes.add(new DeobfuscatorVisitor());
		passes.add(new SourceFileRename());
		passes.add(new RenameVisitor());
		passes.add(new SaveDeobfMapping());

		passes.add(new UsageInfoVisitor());
		passes.add(new CollectConstValues());
		passes.add(new ProcessAnonymous());
		passes.add(new ProcessMethodsForInline());
		return passes;
	}

	public static List<IDexTreeVisitor> getRegionsModePasses(JadxArgs args) {
		List<IDexTreeVisitor> passes = new ArrayList<>();
		// instructions IR
		passes.add(new CheckCode());
		if (args.isDebugInfo()) {
			passes.add(new DebugInfoAttachVisitor());
		}
		passes.add(new AttachTryCatchVisitor());
		if (args.getCommentsLevel() != CommentsLevel.NONE) {
			passes.add(new AttachCommentsVisitor());
		}
		passes.add(new AttachMethodDetails());
		passes.add(new ProcessInstructionsVisitor());

		// blocks IR
		passes.add(new BlockSplitter());
		passes.add(new BlockProcessor());
		passes.add(new BlockFinisher());
		if (args.isRawCFGOutput()) {
			passes.add(DotGraphVisitor.dumpRaw());
		}

		passes.add(new SSATransform());
		passes.add(new MoveInlineVisitor());
		passes.add(new ConstructorVisitor());
		passes.add(new InitCodeVariables());
		if (args.isExtractFinally()) {
			passes.add(new MarkFinallyVisitor());
		}
		passes.add(new ConstInlineVisitor());
		passes.add(new TypeInferenceVisitor());
		if (args.isDebugInfo()) {
			passes.add(new DebugInfoApplyVisitor());
		}
		passes.add(new FixTypesVisitor());
		passes.add(new FinishTypeInference());

		if (args.getUseKotlinMethodsForVarNames() != JadxArgs.UseKotlinMethodsForVarNames.DISABLE) {
			passes.add(new ProcessKotlinInternals());
		}
		passes.add(new CodeRenameVisitor());
		if (args.isInlineMethods()) {
			passes.add(new InlineMethods());
		}
		passes.add(new GenericTypesVisitor());
		passes.add(new ShadowFieldVisitor());
		passes.add(new DeboxingVisitor());
		passes.add(new AnonymousClassVisitor());
		passes.add(new ModVisitor());
		passes.add(new CodeShrinkVisitor());
		passes.add(new ReplaceNewArray());
		if (args.isCfgOutput()) {
			passes.add(DotGraphVisitor.dump());
		}

		// regions IR
		passes.add(new RegionMakerVisitor());
		passes.add(new IfRegionVisitor());
		if (args.isRestoreSwitchOverString()) {
			passes.add(new SwitchOverStringVisitor());
		}
		passes.add(new ReturnVisitor());
		passes.add(new CleanRegions());

		passes.add(new CodeShrinkVisitor());
		passes.add(new MethodInvokeVisitor());
		passes.add(new SimplifyVisitor());
		passes.add(new CheckRegions());

		passes.add(new EnumVisitor());
		passes.add(new FixSwitchOverEnum());
		passes.add(new NonFinalResIdsVisitor());
		passes.add(new ExtractFieldInit());
		passes.add(new FixAccessModifiers());
		passes.add(new ClassModifier());
		passes.add(new LoopRegionVisitor());

		if (args.isInlineMethods()) {
			passes.add(new MarkMethodsForInline());
		}
		passes.add(new ProcessVariables());
		passes.add(new PrepareForCodeGen());
		if (args.isCfgOutput()) {
			passes.add(DotGraphVisitor.dumpRegions());
		}
		return passes;
	}

	public static List<IDexTreeVisitor> getSimpleModePasses(JadxArgs args) {
		List<IDexTreeVisitor> passes = new ArrayList<>();
		if (args.isDebugInfo()) {
			passes.add(new DebugInfoAttachVisitor());
		}
		passes.add(new AttachTryCatchVisitor());
		if (args.getCommentsLevel() != CommentsLevel.NONE) {
			passes.add(new AttachCommentsVisitor());
		}
		passes.add(new AttachMethodDetails());
		passes.add(new ProcessInstructionsVisitor());

		passes.add(new BlockSplitter());
		if (args.isRawCFGOutput()) {
			passes.add(DotGraphVisitor.dumpRaw());
		}
		passes.add(new MethodVisitor("DisableBlockLock", mth -> mth.add(AFlag.DISABLE_BLOCKS_LOCK)));
		passes.add(new BlockProcessor());
		passes.add(new SSATransform());
		passes.add(new MoveInlineVisitor());
		passes.add(new ConstructorVisitor());
		passes.add(new InitCodeVariables());
		passes.add(new ConstInlineVisitor());
		passes.add(new TypeInferenceVisitor());
		if (args.isDebugInfo()) {
			passes.add(new DebugInfoApplyVisitor());
		}
		passes.add(new FixTypesVisitor());
		passes.add(new FinishTypeInference());
		passes.add(new CodeRenameVisitor());
		passes.add(new DeboxingVisitor());
		passes.add(new ModVisitor());
		passes.add(new CodeShrinkVisitor());
		passes.add(new ReplaceNewArray());
		passes.add(new SimplifyVisitor());
		passes.add(new MethodVisitor("ForceGenerateAll", mth -> mth.remove(AFlag.DONT_GENERATE)));
		if (args.isCfgOutput()) {
			passes.add(DotGraphVisitor.dump());
		}
		return passes;
	}

	public static List<IDexTreeVisitor> getFallbackPassesList() {
		List<IDexTreeVisitor> passes = new ArrayList<>();
		passes.add(new AttachTryCatchVisitor());
		passes.add(new AttachCommentsVisitor());
		passes.add(new ProcessInstructionsVisitor());
		passes.add(new FallbackModeVisitor());
		return passes;
	}

	public static final String VERSION_DEV = "dev";

	private static String version;

	public static String getVersion() {
		if (version == null) {
			version = searchJadxVersion();
		}
		return version;
	}

	public static boolean isDevVersion() {
		return getVersion().equals(VERSION_DEV);
	}

	private static String searchJadxVersion() {
		try {
			ClassLoader classLoader = Jadx.class.getClassLoader();
			if (classLoader != null) {
				Enumeration<URL> resources = classLoader.getResources("META-INF/MANIFEST.MF");
				while (resources.hasMoreElements()) {
					try (InputStream is = resources.nextElement().openStream()) {
						Manifest manifest = new Manifest(is);
						String ver = manifest.getMainAttributes().getValue("jadx-version");
						if (ver != null) {
							return ver;
						}
					}
				}
			}
		} catch (Exception e) {
			LOG.error("Can't get manifest file", e);
		}
		return VERSION_DEV;
	}
}
"""

In [ ]:
# Hacer la predicción para un nombre de clase
predicted_name = generate_suggested_name(code, type_label="class")
print("Nombre sugerido:", predicted_name)

Nombre sugerido: getDecompilationMode


In [6]:
import pandas as pd

df = pd.read_csv("./dataset/dataset_final.csv")
df.head()

,code,suggested_name,type,is_correct
0,public class ListManager {\n \n public s...,ListManager,Clase,True
1,public class Main {\n \n public static v...,Main,Clase,False
2,public class Main {\n \n public static v...,CustomLinkedList,Clase,True
3,public class Main {\n \n public static v...,MyList,Clase,False
4,public class Main {\n \n public static v...,appendToEnd,Clase,True


In [13]:
set(df["type"].values.tolist())

{'Clase', 'Métodos', 'Variables'}

In [10]:
# Hacer la predicción para un nombre de clase
predicted_name = generate_suggested_name(code, type_label="Métodos")
print("Nombre sugerido:", predicted_name)

Nombre sugerido: getNormalizedModePasses


In [ ]:
# Hacer la predicción para un nombre de clase
predicted_name = generate_suggested_name(code, type_label="Variables")
print("Nombre sugerido:", predicted_name)

Nombre sugerido: getDecompilationMode
